In [2]:
setwd(paste0(Sys.getenv('R_SOURCES'), '/house_prices'))
source('main.R')
house_prices$helpers$import_libs()

combined_dataset <-
    house_prices$helpers$load_data() %>%
    (house_prices$outliers$remove_outliers) %>%
    (house_prices$missing$fix_all) %>%
    select(-Id)

test_that("should be no NA values except SalePrice column", {
    expect_equal(sum(is.na(combined_dataset %>% select(-SalePrice))), 0)
})

In [8]:
combined_dataset %>%
select(-SalePrice) %>%
select_if(is.numeric) %>%
gather(var, x) %>%
filter(!is.na(x)) %>%
mutate(
    log = log(x + 1),
    sqrt = sqrt(x)
) -> df1

df1 %>% head

var,x,log,sqrt
LotFrontage,65,4.189655,8.062258
LotFrontage,80,4.394449,8.944272
LotFrontage,68,4.234107,8.246211
LotFrontage,60,4.110874,7.745967
LotFrontage,84,4.442651,9.165151
LotFrontage,85,4.454347,9.219544


In [61]:
df1 %>%
gather(tran, value, -var) %>%
group_by(var, tran) %>%
mutate(
    value_normed = (value - mean(value)) / sd(value)
) %>%
select(-value) %>%
group_by(var, tran, value_normed) %>%
arrange(var, tran, value_normed) %>%
summarise(
    k = n()
) %>%
mutate(
    empirical = cumsum(k) / sum(k),
    theoretical = pnorm(value_normed),
    diff_L2 = k*(empirical - theoretical)**2
) %>%
group_by(var, tran) %>%
summarise(
    L2_distance = sum(diff_L2)
) -> df2

df2 %>% head

var,tran,L2_distance
BedroomAbvGr,log,104.29266
BedroomAbvGr,sqrt,111.96468
BedroomAbvGr,x,134.35462
BsmtFinSF1,log,95.80909
BsmtFinSF1,sqrt,49.92343
BsmtFinSF1,x,30.47639


In [71]:
df2 %>%
group_by(var) %>%
nest %>%
mutate(
    best_tran = map(data, function(df) {
        best <- df %>% arrange(L2_distance) %>% head(1)
        vanilla <- df %>% filter(tran == 'x')
        progress_score <- 100 * (vanilla$L2_distance - best$L2_distance) / vanilla$L2_distance
        data_frame(tran = best$tran, progress_score = progress_score)
    })
) %>%
select(var, best_tran) %>%
unnest(best_tran) %>%
filter(tran != 'x') %>%
arrange(desc(progress_score)) -> df3

df3

var,tran,progress_score
GrLivArea,log,91.15794157
X1stFlrSF,log,90.77165686
BsmtUnfSF,sqrt,79.97603070
LotArea,log,73.38993454
TotRmsAbvGrd,log,30.68501506
OverallQual,log,29.94435652
GarageCars,log,24.95676474
BedroomAbvGr,log,22.37508601
OverallCond,log,16.24723935
FullBath,sqrt,4.71886362
